## Signal Amplification by Reversible Exchange (SABRE) - Pyridine: Three Spins

This example demonstrates a simple SABRE simulation of pyridine. Key details of the simulation are as follows:

- **Spin System**: The system consists of two hydride protons and one ligand proton.
- **Simplifications**: Chemical exchange and relaxation effects are not included in this simulation.
- **Simulation Conditions**: The simulation is performed near the energy level anticrossing field, leading to coherent oscillations between quantum states.
- **Performance**: The example runs in a few seconds on a laptop with an 11th-generation i5 processor.

This example provides an accessible introduction to SABRE simulations while focusing on the essential dynamics of the spin system.

### Imports

- **NumPy**: Used for creating and manipulating arrays.
- **Matplotlib**: Used for plotting data and visualizing results.
- **Spinguin**: Provides the core functionality for simulating spin systems, including Hamiltonian calculations, propagators, and measurements.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import spinguin as sg

### Simulation Parameters

The following simulation parameters are defined:

- **Magnetic Field (T)**: This is a global parameter in the Spinguin package. It is set to 5.3 mT.
- **Time Step (s)**: Set to 1 ms.
- **Number of Time Steps**: Set to 1000, resulting in a total simulation time of 1 second.

In [ ]:
# Define global simulation parameters
sg.parameters.magnetic_field = 5.3e-3

# Define the evolution parameters
dt = 1e-3
N_steps = 1000

### Define the Spin System

To perform the simulations, we need to define the spin system and basis set.

- **Spin System**: First, the spin system is initialized by defining the isotopes.
- **Maximum Spin Order**: The simulations are performed in Liouville space in the spherical tensor basis set. Because the spin system is small, the maximum spin order is set to the number of spins in the system (no approximation).
- **Build the Basis Set**: Finally, the basis set is built using the `build()` method.

In [ ]:
# Define the spin system and basis set
spin_system = sg.SpinSystem(['1H', '1H', '1H'])
spin_system.basis.max_spin_order = 3
spin_system.basis.build()

### Define the NMR Interactions

We simulate only the coherent spin dynamics, which involves the definition of the following NMR interactions:

- **Chemical Shifts**: Specified in parts per million (ppm).
- **Scalar Couplings**: Specified in Hertz (Hz).

These parameters are set as attributes to the previously created `SpinSystem` object.

In [ ]:
# Define the NMR interactions
spin_system.chemical_shifts = [-22.7, -22.7, 8.34]
spin_system.J_couplings = [
    [ 0,     0,      0],
    [-6.53,  0,      0],
    [ 0.00,  1.66,   0]
]

## Calculate the Hamiltonian, Liouvillian, and the Time Propagator

The following steps are performed in this section:

- **Hamiltonian**: The Hamiltonian is calculated for the defined spin system.
- **Liouvillian**: In the Spinguin package, the Liouvillian is defined as $L = -iH - R + K$. We use the inbuilt function to calculate the Liouvillian using the correct definition. $R$ and $K$ are not given, as the simulation involves only the coherent dynamics.
- **Time Propagator**: Using the Liouvillian and the defined time step, the time propagator is computed. This propagator will be used to evolve the spin system over time.

In [ ]:
# Calculate the Hamiltonian, Liouvillian, and the Time Propagator for the spin system
H = sg.hamiltonian(spin_system)
L = sg.liouvillian(H)
P = sg.propagator(L, dt)

### Assign the Initial State

The initial state for this simulation is defined as follows:

- **Hydride Spins**: The hydride spins are initialized in a singlet state.
- **Substrate Spin**: The substrate spin is assumed to be in "thermal equilibrium", i.e., in the unit state, as the magnetization is negligible at the small magnetic field.

In [ ]:
# Initialize the density matrix
rho = sg.singlet_state(spin_system, 0, 1)

### Create an Empty Array for Storing Magnetizations

To store the magnetizations during the simulation, we need to create an empty array.

- The simulation involves 1000 time steps.
- After each time step, the magnetization of each spin will be calculated.
- The array will store the magnetizations for all spins at each time step.

In [ ]:
# Create an empty array to store magnetizations
magnetizations = np.empty((spin_system.nspins, N_steps), dtype=complex)

### Perform the Time Evolution

- Loop over the defined number of time steps.
- Calculate the magnetization for each spin at each time step.
- Propagate the spin system forward in time during each step using the time propagator.
- Save the calculated magnetizations to the previously created array for later analysis.

In [ ]:
# Perform the time evolution of the spin system
for step in range(N_steps):

    # Measure the magnetization of each spin and store it in the magnetizations array
    for spin in range(spin_system.nspins):
        magnetizations[spin, step] = sg.measure(spin_system, rho, f"I(z,{spin})")

    # Propagate the density matrix to the next time step
    rho = P @ rho

### Plot the Magnetizations and Visualize the Results

- Calculate the time axis for the simulation.
- Plot the magnetizations of each spin as a function of time.
- Observe that the magnetizations start to oscillate coherently. Near maximal polarization of the substrate spin takes place after approximately 0.5 seconds.

In [ ]:
# Create a time axis for the simulation
t = sg.time_axis(N_steps, dt)

# Plot the magnetizations for each spin as a function of time
for spin in range(spin_system.nspins):
    plt.plot(t, np.real(magnetizations[spin]), label=f"Spin {spin+1}")

# Add a legend to identify each spin
plt.legend(loc="upper right")

# Add labels and title to the plot for clarity
plt.xlabel("Time (s)")
plt.ylabel("Magnetization")
plt.title("SABRE-Hyperpolarization of Pyridine")

# Adjust layout to prevent overlapping elements and display the plot
plt.tight_layout()
plt.show()

# Clear the figure to avoid overlapping plots in subsequent cells
plt.clf()